Dive deep into the fascinating world of Crew AI and learn how to build your autonomous agents from scratch. This project is a comprehensive guide to understanding and implementing Crew AI's capabilities to design intelligent systems that can perform tasks autonomously, think for themselves, and work together as a crew.



In [1]:
!pip install crewai
!pip install tools
!pip install exa_py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 5.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.9/642.9 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.9/19.9 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.9/160.9 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.1/250

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "" # Replace with your OpenAI API key

In [4]:
from textwrap import dedent
from crewai import Agent

In [5]:
class MeetingPrepAgents():
    def research_agent(self):
      return Agent(
        role="Research Specialist",
        goal='Conduct thorough research on people and companies involved in the meeting',
        tools=ExaSearchToolset.tools(),
        backstory=dedent("""\
          As a Research Specialist, your mission is to uncover detailed information
					about the individuals and entities participating in the meeting. Your insights
					will lay the groundwork for strategic meeting preparation."""),
        verbose=True
      )

    def industry_analysis_agent(self):
      return Agent(
        role='Industry Analyst',
        goal='Analyze the current industry trends, challenges, and opportunities',
        tools=ExaSearchToolset.tools(),
        backstory=dedent("""\
            As an Industry Analyst, your analysis will identify key trends,
            challenges facing the industry, and potential opportunities that
            could be leveraged during the meeting for strategic advantage."""),
        verbose=True
      )

    def meeting_strategy_agent(self):
      return Agent(
        role='Meeting Strategy Advisor',
        goal='Develop talking points, questions, and strategic angles for the meeting',
        backstory=dedent("""\
            As a Strategy Advisor, your expertise will guide the development of
            talking points, insightful questions, and strategic angles
            to ensure the meeting's objectives are achieved."""),
        verbose=True
      )

    def summary_and_briefing_agent(self):
      return Agent(
        role='Briefing Coordinator',
        goal='Compile all gathered information into a concise, informative briefing document',
        backstory=dedent("""\
            As the Briefing Coordinator, your role is to consolidate the research,
            analysis, and strategic insights."""),
        verbose=True
      )

In [6]:
from textwrap import dedent
from crewai import Task

In [7]:
class MeetingPrepTasks():
  def research_task(self, agent, meeting_participants, meeting_context):
    return Task(
      description=dedent(f"""\
        Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {meeting_participants}
				Meeting Context: {meeting_context}"""),
      expected_output=dedent(f"""\
        A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
      agent=agent,
      async_execution=True
    )

  def industry_analysis_task(self, agent, meeting_participants, meeting_context):
    return Task(
			description=dedent(f"""\
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {meeting_participants}
				Meeting Context: {meeting_context}"""),
			expected_output=dedent("""\
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
		)

  def meeting_strategy_task(self, agent, meeting_context, meeting_objective):
    return Task(
			description=dedent(f"""\
				Develop strategic talking points, questions, and discussion angles
				for the meeting based on the research and industry analysis conducted

				Meeting Context: {meeting_context}
				Meeting Objective: {meeting_objective}"""),
			expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
		)

  def summary_and_briefing_task(self, agent, meeting_context, meeting_objective):
    return Task(
			description=dedent(f"""\
				Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {meeting_context}
				Meeting Objective: {meeting_objective}"""),
			expected_output=dedent("""\
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent
		)

In [8]:
import os
from exa_py import Exa
from langchain.agents import tool

class ExaSearchToolset():
  @tool
  def search(query: str):
    """Search for a webpage based on the query."""
    return ExaSearchToolset._exa().search(f"{query}", use_autoprompt=True, num_results=3)

  @tool
  def find_similar(url: str):
      """Search for webpages similar to a given URL.
      The url passed in should be a URL returned from `search`.
      """
      return ExaSearchToolset._exa().find_similar(url, num_results=3)

  @tool
  def get_contents(ids: str):
      """Get the contents of a webpage.
      The ids must be passed in as a list, a list of ids returned from `search`.
      """
      ids = eval(ids)

      contents = str(ExaSearchToolset._exa().get_contents(ids))
      contents = contents.split("URL:")
      contents = [content[:1000] for content in contents]
      return "\n\n".join(contents)

  def tools():
    return [
      ExaSearchToolset.search,
      ExaSearchToolset.find_similar,
      ExaSearchToolset.get_contents
    ]


  def _exa():
    return Exa(api_key=os.environ.get('EXA_API_KEY'))

ImportError: cannot import name 'tool' from 'langchain.agents' (/usr/local/lib/python3.12/dist-packages/langchain/agents/__init__.py)

In [ ]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MeetingPrepTasks
from agents import MeetingPrepAgents


def main():
    load_dotenv()

    print("## Welcome to the Meeting Prep Crew")
    print('-------------------------------')
    meeting_participants = input("What are the emails for the participants (other than you) in the meeting?\n")
    meeting_context = input("What is the context of the meeting?\n")
    meeting_objective = input("What is your objective for this meeting?\n")

    tasks = MeetingPrepTasks()
    agents = MeetingPrepAgents()

    # create agents
    research_agent = agents.research_agent()
    industry_analysis_agent = agents.industry_analysis_agent()
    meeting_strategy_agent = agents.meeting_strategy_agent()
    summary_and_briefing_agent = agents.summary_and_briefing_agent()

    # create tasks
    research_task = tasks.research_task(research_agent, meeting_participants, meeting_context)
    industry_analysis_task = tasks.industry_analysis_task(industry_analysis_agent, meeting_participants, meeting_context)
    meeting_strategy_task = tasks.meeting_strategy_task(meeting_strategy_agent, meeting_context, meeting_objective)
    summary_and_briefing_task = tasks.summary_and_briefing_task(summary_and_briefing_agent, meeting_context, meeting_objective)

    meeting_strategy_task.context = [research_task, industry_analysis_task]
    summary_and_briefing_task.context = [research_task, industry_analysis_task, meeting_strategy_task]

    crew = Crew(
      agents=[
        research_agent,
        industry_analysis_agent,
        meeting_strategy_agent,
        summary_and_briefing_agent
      ],
      tasks=[
        research_task,
        industry_analysis_task,
        meeting_strategy_task,
        summary_and_briefing_task
      ]
    )

    result = crew.kickoff()

    print(result)

if __name__ == "__main__":
    main()